In [27]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/carstest/cars_test.csv
/kaggle/input/carstrain/cars_train.csv


In [50]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Carregar dataset de treinamento
df_train = pd.read_csv('/kaggle/input/carstrain/cars_train.csv', encoding='utf-16le')

# Carregar dataset de teste
df_test = pd.read_csv('/kaggle/input/carstest/cars_test.csv', encoding='utf-16le')

# Especificar separador de colunas ao ler o arquivo CSV
df_train = pd.read_csv('/kaggle/input/carstrain/cars_train.csv', encoding='utf-16le', delimiter='\t')

# Substituir o separador de colunas por vírgula (',')
df_train.to_csv('dataset_modificado_cars_test.csv', index=False, sep=',')

# Verificar se o novo arquivo foi criado corretamente
new_data = pd.read_csv('dataset_modificado_cars_test.csv')
#print(new_data.head())

df_train.head() # Exibe as primeiras linhas do conjunto de treinamento


,id,num_fotos,marca,modelo,versao,ano_de_fabricacao,ano_modelo,hodometro,cambio,num_portas,...,elegivel_revisao,dono_aceita_troca,veiculo_único_dono,revisoes_concessionaria,ipva_pago,veiculo_licenciado,garantia_de_fábrica,revisoes_dentro_agenda,veiculo_alienado,preco
0,300716223898539419613863097469899222392,8.0,NISSAN,KICKS,1.6 16V FLEXSTART SL 4P XTRONIC,2017,2017.0,67772.0,CVT,4,...,False,NaN,NaN,Todas as revisões feitas pela concessionária,IPVA pago,Licenciado,NaN,NaN,NaN,74732.590084
1,279639842134129588306469566150288644214,8.0,JEEP,COMPASS,2.0 16V FLEX LIMITED AUTOMÁTICO,2017,2017.0,62979.0,Automática,4,...,False,Aceita troca,NaN,NaN,IPVA pago,NaN,NaN,NaN,NaN,81965.332634
2,56414460810621048900295678236538171981,16.0,KIA,SORENTO,2.4 16V GASOLINA EX 7L AWD AUTOMÁTICO,2018,2019.0,44070.0,Automática,4,...,False,Aceita troca,NaN,NaN,NaN,NaN,NaN,NaN,NaN,162824.814472
3,56862509826849933428086372390159405545,14.0,VOLKSWAGEN,AMAROK,2.0 HIGHLINE 4X4 CD 16V TURBO INTERCOOLER DIES...,2013,2015.0,85357.0,Automática,4,...,False,Aceita troca,NaN,NaN,IPVA pago,Licenciado,NaN,NaN,NaN,123681.358857
4,338980975753200343894519909855598027197,8.0,SSANGYONG,KORANDO,2.0 GLS 4X4 16V TURBO DIESEL 4P AUTOMÁTICO,2013,2015.0,71491.0,Automática,4,...,False,NaN,NaN,Todas as revisões feitas pela concessionária,NaN,NaN,Garantia de fábrica,Todas as revisões feitas pela agenda do carro,NaN,82419.763891


In [44]:
# Separar features (X) e target (y) do conjunto de treinamento
X_train = df_train.drop('preco', axis=1)
y_train = df_train['preco']


In [45]:
# Realizar One-Hot Encoding nas features categóricas do conjunto de treinamento
X_train_encoded = pd.get_dummies(X_train)


In [46]:
# Realizar One-Hot Encoding nas features categóricas do conjunto de teste
X_test_encoded = pd.get_dummies(df_test)

In [32]:
# Concatenar os dataframes de treinamento e teste para garantir a correspondência das colunas
X_train_encoded, X_test_encoded = X_train_encoded.align(X_test_encoded, join='left', axis=1, fill_value=0)

In [42]:
# Preencher valores ausentes (NaN) usando a média das colunas
X_train_encoded = X_train_encoded.fillna(X_train_encoded.mean())
X_test_encoded = X_test_encoded.fillna(X_train_encoded.mean())  # Preencher com a média do conjunto de treinamento



KeyboardInterrupt: 

In [56]:
# Análise das principais estatísticas da base de dados
# Estatísticas descritivas das variáveis numéricas
numeric_stats = X_train_encoded.describe()
print("Estatísticas descritivas das variáveis numéricas:")
print(numeric_stats)

Estatísticas descritivas das variáveis numéricas:
          num_fotos  ano_de_fabricacao    ano_modelo      hodometro  \
count  29407.000000       29584.000000  29584.000000   29584.000000   
mean      10.323834        2016.758552   2017.808985   58430.592077   
std        3.487334           4.062422      2.673930   32561.769309   
min        8.000000        1985.000000   1997.000000     100.000000   
25%        8.000000        2015.000000   2016.000000   31214.000000   
50%        8.000000        2018.000000   2018.000000   57434.000000   
75%       14.000000        2019.000000   2020.000000   81953.500000   
max       21.000000        2022.000000   2023.000000  390065.000000   

         num_portas  veiculo_alienado  \
count  29584.000000               0.0   
mean       3.940677               NaN   
std        0.338360               NaN   
min        2.000000               NaN   
25%        4.000000               NaN   
50%        4.000000               NaN   
75%        4.000000    

In [ ]:
# Plote histogramas para visualizar a distribuição das variáveis numéricas
df.hist(figsize=(12, 8))
plt.tight_layout()
plt.show()

/tmp/ipykernel_32/509851193.py:3: UserWarning: Tight layout not applied. tight_layout cannot make axes width small enough to accommodate all axes decorations
  plt.tight_layout()


In [ ]:
# Plote box plots para identificar possíveis outliers nas variáveis numéricas
df.boxplot(figsize=(12, 8))
plt.tight_layout()
plt.show()

In [54]:
# Calcular contagem e frequência das variáveis categóricas
categorical_stats = {}
for column in df_train.select_dtypes(include='object'):
    counts = df_train[column].value_counts()
    frequencies = counts / counts.sum()
    categorical_stats[column] = pd.concat([counts, frequencies], axis=1, keys=['Contagem', 'Frequência'])


In [55]:
print("Estatísticas descritivas das variáveis categóricas:")
for column, stats in categorical_stats.items():
    print(f"\n{column}:\n{stats}")

Estatísticas descritivas das variáveis categóricas:

id:
                                         Contagem  Frequência
300716223898539419613863097469899222392         1    0.000034
285890682893499494121046105359660467160         1    0.000034
164151341442204939739633671446766506660         1    0.000034
104605966898056711399437656393867106581         1    0.000034
225802387330645423199512943057734533233         1    0.000034
...                                           ...         ...
51948056493465724900513443742353763328          1    0.000034
56763651987108516122847468318822603887          1    0.000034
301609676912075371890494688957843480392         1    0.000034
31630061149326072384596052351170200337          1    0.000034
171781413417552881896009715888472310172         1    0.000034

[29584 rows x 2 columns]

marca:
               Contagem  Frequência
VOLKSWAGEN         4594    0.155287
CHEVROLET          3020    0.102082
TOYOTA             2180    0.073688
HYUNDAI            20

In [ ]:
# Relatório com análise das principais estatísticas da base de dados
print("\nRelatório: Análise das principais estatísticas da base de dados")


In [ ]:
# Estatísticas descritivas das variáveis numéricas
print("\nEstatísticas descritivas das variáveis numéricas:")
print(numeric_stats)

In [ ]:
# Comentário: As estatísticas descritivas das variáveis numéricas fornecem informações importantes sobre a distribuição, centralidade e dispersão dos dados. A média indica o valor médio dos dados, o desvio padrão mede a dispersão em relação à média, o mínimo e o máximo mostram os valores extremos dos dados, e os quartis (25%, 50% e 75%) revelam a distribuição dos dados em termos de percentis.

In [ ]:
# Gráficos para visualizar a distribuição das variáveis numéricas e identificar possíveis outliers
print("\nGráficos para visualizar a distribuição das variáveis numéricas e identificar possíveis outliers:")
X_train_encoded.hist(figsize=(12, 8))
plt.tight_layout()
plt.show()

X_train_encoded.boxplot(figsize=(12, 8))
plt.tight_layout()
plt.show()

In [ ]:
# Comentário: Os histogramas ajudam a visualizar a forma e a distribuição dos dados numéricos, permitindo identificar se seguem uma distribuição normal ou apresentam assimetria. Os box plots fornecem informações sobre a mediana, os quartis, a dispersão e a presença de possíveis outliers nos dados.


In [ ]:
# Estatísticas descritivas das variáveis categóricas
print("\nEstatísticas descritivas das variáveis categóricas:")
for column, stats in categorical_stats.items():
    print(f"\n{column}:\n{stats}")

In [ ]:
# Comentário: As estatísticas descritivas das variáveis categóricas mostram a contagem e a frequência de cada categoria, fornecendo insights sobre a distribuição das categorias